# BCGW Service URL Updater - October 20th 2021
DataBC has been working to improve web service performance and stability for public data access web services available to ArcGIS Online via BCs Map Hub. Historically we have published a single large service with over 700 layers. The single service will be replaced by multiple smaller services published by schema. Duplicate Map Image Layer items will also be deprecated in favour of the corresponding authoritative Feature Layer items. We are now ready to implement the changes.

#### Michael Dykes (Michael.Dykes@gov.bc.ca)     ***Please contact me incase of bugs/errors/issues

## Instructions

#### 1. To start select "Restart & Run All" from the Kernel menu. This will run all the code in the notebook and the GUI interface will appear at the bottom.

![text](https://github.com/MichaelDykesBC/AGO/blob/master/ChangeOver_Instruct1.PNG?raw=true)

#### 2. While the code is running the tag next to the code should have a "*" symbol indicating that the code is running. On successful completion this "*" should change to a number.

![text](https://github.com/MichaelDykesBC/AGO/blob/master/ChangeOver_Instruct2.PNG?raw=true)

#### 3. The item/group selection pane will default to content you own (only "Web Map","Web Mapping Application","StoryMap","Dashboard" items are displayed). You can select a group from the dropdown to load items from that particular group. Hitting the "All My Content" button will return to a list of only items you own.

![text](https://github.com/MichaelDykesBC/AGO/blob/master/ChangeOver_Instruct3.PNG?raw=true)

#### 4. On selection of an item, the item pane should update with information about that item (Title, Type, Owner, Creation Date, Modified Date) with links to the item page and the option to open the item in its viewer. Click the "Check Item" button to perform a search of an item's JSON content for old BCGW Service URLs for updating.

![text](https://github.com/MichaelDykesBC/AGO/blob/master/ChangeOver_Instruct4.PNG?raw=true)

#### 5. On completion of the BCGW Service URL check you will either get a message that no updates are needed or a list of layers that need updating. In the layer list you will see the layer name, the existing BCGW URL value, the suggested value from the DataBC lookup table, or have the option to override the suggested value with your own text entry (if you leave it blank it will accept the suggested value from DataBC).

![text](https://github.com/MichaelDykesBC/AGO/blob/master/ChangeOver_Instruct8.PNG?raw=true)
![text](https://github.com/MichaelDykesBC/AGO/blob/master/ChangeOver_Instruct5.PNG?raw=true)

#### 6. To update the layers click the "Update All URLs" button on the bottom of the list. If the item is updated successfully you will get a message indicating this. If it fails you will also get an indication (probably a bunch of Python errors).

![text](https://github.com/MichaelDykesBC/AGO/blob/master/ChangeOver_Instruct6.PNG?raw=true)
![text](https://github.com/MichaelDykesBC/AGO/blob/master/ChangeOver_Instruct7.PNG?raw=true)

#### 7. Incase you need to restore the original JSON, you can click the "Restore Item JSON" button, which will restore the JSON from a file located in your AGO Notebook File Directory (not your AGO Content). You can also download the backup JSON files from here (they are named by itemID).

![text](https://github.com/MichaelDykesBC/AGO/blob/master/ChangeOver_Instruct9.PNG?raw=true)
![text](https://github.com/MichaelDykesBC/AGO/blob/master/ChangeOver_Instruct10.PNG?raw=true)
![text](https://github.com/MichaelDykesBC/AGO/blob/master/ChangeOver_Instruct11.PNG?raw=true)

## The Code

### 1. Import Required Libraries/Modules and ArcGIS Online Connection

In [1]:
import os, getpass, json, time, base64, IPython, random
from arcgis.gis import GIS
from functools import reduce
from operator import getitem
import ipywidgets as widgets
import pandas as pd

gis = GIS('home')
AGO_Max_Items = 5000

/opt/conda/lib/python3.7/site-packages/arcgis/gis/__init__.py:430: UserWarning: You are logged on as michael.dykes_bcgov03 with an administrator role, proceed with caution.
  self.users.me.username)


### 2. Set File Folder Structure for JSON Text File Backups

In [2]:
#Folder structure in AGO Notebooks Files Directory to hold JSON backup files (for quick restoration of previous JSON values)
WorkFolderPath = '/arcgis/home/BCGW_Updates'
BackupPath = WorkFolderPath + "/AGO_JSON_Backups"
# If folders don't exist, create them
if not os.path.exists(WorkFolderPath):
    os.mkdir(WorkFolderPath)
    os.mkdir(BackupPath)

### 3. Load Functions

In [3]:
# Set JSON values in nested dictionaries
def Set_Nested_JSON_Value(dataDict, mapList, val_url, val_itemid):
    # Set the JSON layer URL references to new value (from lookup table/GUI)
    reduce(getitem, mapList[:-1], dataDict)[mapList[-1]] = val_url
    # If new itemId value has been supplied
    if val_itemid:
        # Check if itemId is a key in the dictionary associated with the layer
        if 'itemId' in reduce(getitem, mapList[:-1], dataDict):
            # Set the JSON layer URL references to new value (from lookup table/GUI)
            reduce(getitem, mapList[:-1], dataDict)['itemId'] = val_itemid
    return dataDict

# Search nested dictionaries for JSON values, store the "path" to get there and the value itself
def Search_JSON(storage, haystack, needle, path=None):
    storage = storage
    if path is None:
        path = []
    if isinstance(haystack, dict):
        if needle in haystack:
            if haystack[needle]:
                if "mpcm/bcgwpub" in haystack[needle] or "mpcm/bcgw" in haystack[needle]:
                    path.append(needle)
                    toappend = path,haystack[needle]
                    storage.append(toappend)
        for k, v in haystack.items():
            Search_JSON(storage, v, needle, path + [k])
    elif isinstance(haystack, list):
        for idx, v in enumerate(haystack):
            Search_JSON(storage, v, needle, path + [idx])
            
# Search the JSON for itemId, url, and baseURL keys and put the JSON 'path' and value in ReferenceList            
def Check_JSON_ItemReferences(ItemID):
    ReferenceList = []
    item = gis.content.get(ItemID)
    try:
        item_data = item.get_data()
        Search_JSON(ReferenceList,item_data,"url")
    except:
        pass
    return ReferenceList
        
# Backup JSON data into text files on your harddrive (to be extra careful) you can use AGO assistant to 
# copy and paste them back into your AGO JSON if something goes wrong    
def Create_JSON_BackupFile(ItemID):
    item = gis.content.get(ItemID)
    item_data = item.get_data()    
    with open(BackupPath + "/" + ItemID + "_Backup.json", 'w') as outfile:
        json.dump(item_data, outfile)

# Restore item JSON from backup
def restore_button_click(self):
    bottom_box.children = ()
    backupfile = BackupPath + "/" + SearchDict[itemSelection.value] + "_Backup.json"
    if os.path.isfile(backupfile):
        with open(backupfile) as json_file:
            backupjson = json.load(json_file)
            item = gis.content.get(SearchDict[itemSelection.value])
            item.update(data=backupjson)
            BackupPass_text = widgets.Label(value="JSON Data Restored from Backup Successfully")
            bottom_box.children += (BackupPass_text,)     
    else:
        BackupFail_text = widgets.Label(value="No Backup Found")
        bottom_box.children += (BackupFail_text,)  

# Update AGO Item JSON based on GUI list
def update_URL_references(self):
    item = gis.content.get(SearchDict[itemSelection.value])
    item_data = item.get_data()

    Create_JSON_BackupFile(SearchDict[itemSelection.value])
    UpdateList = Checkbutton.UpdateList
    if UpdateList:
        for i in range(1,len(bottom_box.children[1:-1])):
            if bottom_box.children[i].children[1].value == UpdateList[i-1][1]:
                if bottom_box.children[i].children[3].value:
                    Set_Nested_JSON_Value(item_data,UpdateList[i-1][0],bottom_box.children[i].children[3].value, None)
                else:
                    Set_Nested_JSON_Value(item_data,UpdateList[i-1][0],bottom_box.children[i].children[2].value, UpdateList[i-1][2])          
        try:      
            item.update(data=item_data)
            gif_list = ["https://giphy.com/embed/ddHhhUBn25cuQ","https://giphy.com/embed/kyLYXonQYYfwYDIeZl","https://giphy.com/embed/gFi7V9CRBQVW0","https://giphy.com/embed/l0MYt5jPR6QX5pnqM",
                        "https://giphy.com/embed/YTbZzCkRQCEJa","https://giphy.com/embed/s2qXK8wAvkHTO","https://giphy.com/embed/35HTaxVJWzp2QOShct","https://giphy.com/embed/8j3CTd8YJtAv6"]
            random_supportive_gif = widgets.HTML('<iframe src="' + random.choice(gif_list) + '" width="280" height="250" frameBorder="0" class="giphy-embed" allowFullScreen></iframe>')
            Update_text = widgets.Label(value= "AGO Item Updated Successfully!")
            bottom_box.children = (Update_text,random_supportive_gif,) 
        except:
            Update_text = widgets.Label(value= "You Don't Have Permission to Update this Item")
            bottom_box.children = (Update_text,)  
    else:
        UpdateFail_text = widgets.Label(value= "AGO Item Updated Failed")
        bottom_box.children += (UpdateFail_text,)

# Create link to download the table of layers and url references to a csv file on your computer
def create_download_link(title = "Download as CSV file"):
    filename = gis.content.get(SearchDict[itemSelection.value]).title[:50] + "_BCGW_URLUpdate.csv"
    datalist =[]
    for row in bottom_box.children[1:]:
        datalist.append([row.children[0].value,row.children[1].value,row.children[2].value,row.children[3].value])
    df = pd.DataFrame(data = datalist, columns=['LAYERNAME','EXISTING_URL','SUGGESTED_URL','URL_OVERRIDE'])
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return html

# Executes function which populates GUI list of layers and url references (only those with "mpcm/bcgwpub" and "mpcm/bcgw" referenced)
def check_button_click(self):
    bottom_box.children = ()
    self.UpdateList = []
    self.ReferenceList = Check_JSON_ItemReferences(SearchDict[itemSelection.value])
    if self.ReferenceList:
        text_layername = widgets.Label(value="Layer Name")
        text_layername.layout.width = "20%"   
        text_existURL = widgets.Label(value="Existing URL Reference")
        text_existURL.layout.width = "25%"
        text_newURL = widgets.Label(value="Suggested New URL Reference")
        text_newURL.layout.width = "25%"
        text_overrideURL = widgets.Label(value="New URL Suggestion Override")
        text_overrideURL.layout.width = "25%" 
        RowNameLayout = widgets.HBox([text_layername,text_existURL,text_newURL,text_overrideURL],layout=widgets.Layout(width='100%',display='inline-flex',flex_flow='row wrap'))
        bottom_box.children += (RowNameLayout,)
        n = 1
        for row in self.ReferenceList:
            if row[1]:
                oldpath = os.path.join(*os.path.normpath(row[1]).split(os.path.sep)[5:])
                if oldpath in OldUrl_Dict:
                    if isinstance(OldUrl_Dict[oldpath][0],float):
                        value_layername = widgets.HTML(value= '<style>p{word-wrap: break-word}</style> <p>'+ "" +' </p>') 
                    else:
                        value_layername = widgets.HTML(value= '<style>p{word-wrap: break-word}</style> <p>'+ OldUrl_Dict[oldpath][0] +' </p>') 
                        
                    if isinstance(OldUrl_Dict[oldpath][1],float):
                        value_newURL = widgets.Textarea(value="",disabled=True)
                    else:
                        if OldUrl_Dict[oldpath][1] and not isinstance(OldUrl_Dict[oldpath][1],float):
                            if "https://services6.arcgis.com/" not in OldUrl_Dict[oldpath][1]:
                                if "arcserver" in row[1] and "arcserver" in OldUrl_Dict[oldpath][1]: 
                                    value_newURL = widgets.Textarea(value=OldUrl_Dict[oldpath][1],disabled=True)
                                elif "arcserver" in row[1] and "arcserver" not in OldUrl_Dict[oldpath][1]:
                                    value_newURL = widgets.Textarea(value=OldUrl_Dict[oldpath][1].replace("arcgis","arcserver"),disabled=True)
                                elif "arcgis" in row[1] and "arcgis" in OldUrl_Dict[oldpath][1]: 
                                    value_newURL = widgets.Textarea(value=OldUrl_Dict[oldpath][1],disabled=True)
                                elif "arcgis" in row[1] and "arcgis" not in OldUrl_Dict[oldpath][1]:
                                    value_newURL = widgets.Textarea(value=OldUrl_Dict[oldpath][1].replace("arcserver","arcgis"),disabled=True)
                            else:
                                value_newURL = widgets.Textarea(value=OldUrl_Dict[oldpath][1],disabled=True)
                        else:
                            value_newURL = widgets.Textarea(value="",disabled=True)
                else:
                    if "dynamicLayer?layer" in row[1]:
                        layername = row[1][row[1].find("dataSourceName")+len("dataSourceName")+6:row[1].find("dataSourceName")+len("dataSourceName")+6+row[1][row[1].find("dataSourceName")+len("dataSourceName")+6:].find('",%20"')]
                        if layername in WHSE_Dict:
                            OldUrl_Dict[layername] = WHSE_Dict[layername]
                            value_layername = widgets.Textarea(value=layername,disabled=True)
                            value_newURL = widgets.Textarea(value=WHSE_Dict[layername][1],disabled=True) 
                        else:
                            value_layername = widgets.Textarea(value=layername,disabled=True)
                            value_newURL = widgets.Textarea(value="",disabled=True) 
                    else:
                        value_layername = widgets.Textarea(value="",disabled=True)
                        value_newURL = widgets.Textarea(value="",disabled=True)     
                    
                value_layername.layout = widgets.Layout(height='100%', width='15%',display='inline-flex',flex_flow='row wrap')
                value_existURL = widgets.Textarea(value=row[1],disabled=True)
                value_existURL.layout = widgets.Layout(width='25%',display='inline-flex',flex_flow='row wrap')
                value_newURL.layout = widgets.Layout(width='25%',display='inline-flex',flex_flow='row wrap')  
                value_override = widgets.Textarea(value=None)
                value_override.layout = widgets.Layout(width='25%',display='inline-flex',flex_flow='row wrap')
                
                if (n % 2) == 0:
                    boxcolor = 'black'
                else:
                    boxcolor = 'blue'
                
                RowLayout = widgets.HBox([value_layername,value_existURL,value_newURL,value_override],layout=widgets.Layout(width='100%',display='inline-flex',flex_flow='row wrap',border='1px solid ' + boxcolor))
                bottom_box.children += (RowLayout,)
                if "dynamicLayer?layer" in row[1]:
                    toappend = row[0],row[1],""
                else:
                    toappend = row[0],row[1],OldUrl_Dict[oldpath][2]

                self.UpdateList.append(toappend)
                n = n + 1
                
        UpdateAllButton = widgets.Button(description='Update All URLs')
        UpdateAllButton.on_click(update_URL_references)
        link = widgets.HTML(value=create_download_link())
        bottom_box.children += (UpdateAllButton,link,)
        
    if not self.UpdateList:
        NoUpdate_text = widgets.Label(value="No Updates Needed/Found")
        bottom_box.children = (NoUpdate_text,)
        
def group_content_selection_change(self):
    if groupSelection.value:
        groupid = GroupSearchDict[groupSelection.value]
        group = gis.groups.get(groupid)

        itemtypes_list = ["Web Map","Web Mapping Application","StoryMap","Dashboard"]
        new_item_list = []
        for item in group.content():
            if item.type in itemtypes_list:
                new_item_list.append([item.title,item.id])
        sorteditem_list = sorted(new_item_list,key = lambda x:x[0])

        global SearchDict
        SearchDict = {}
        for row in sorteditem_list:
            SearchDisplay = row[0] + " | " + gis.content.get(row[1]).type
            SearchDict[SearchDisplay] = row[1]
        itemSelection.options = sorted(SearchDict.keys())

def own_content_selection(self):
    itemtypes_list = ["Web Map","Web Mapping Application","StoryMap","Dashboard"]
    new_item_list = []
    for item in gis.content.search(query="* AND \  owner:" + gis.users.me.username, max_items=AGO_Max_Items):
        if item.type in itemtypes_list:
            new_item_list.append([item.title,item.id])
    sorteditem_list = sorted(new_item_list,key = lambda x:x[0])
    
    global SearchDict
    SearchDict = {}
    for row in sorteditem_list:
        SearchDisplay = row[0] + " | " + gis.content.get(row[1]).type
        SearchDict[SearchDisplay] = row[1]
    itemSelection.options = sorted(SearchDict.keys())
    groupSelection.value = None
    
# Watches selection list for change in selection and updates the other GUIs in response
def on_selection_change(self):
    if SearchDict:
        bottom_box.children = ()
        title_text.value = "Title: " + gis.content.get(SearchDict[itemSelection.value]).title
        type_text.value = "Type: " + gis.content.get(SearchDict[itemSelection.value]).type
        owner_text.value = "Owner: " + gis.content.get(SearchDict[itemSelection.value]).owner
        created_text.value = "Created: " + str(time.strftime('%Y-%m-%d %H:%M', time.localtime(gis.content.get(SearchDict[itemSelection.value]).created/1000)))
        modified_text.value = "Modified: " + str(time.strftime('%Y-%m-%d %H:%M', time.localtime(gis.content.get(SearchDict[itemSelection.value]).modified/1000)))
        itemurl_text.value = '<a target="_blank" rel="noopener noreferrer" href=' + gis.url + "/home/item.html?id=" + SearchDict[itemSelection.value] + '>Go to Item Page</a>'

        if gis.content.get(SearchDict[itemSelection.value]).type == "Web Map":
            url_text.value = '<a target="_blank" rel="noopener noreferrer" href=' + "https://bcgov03.maps.arcgis.com/home/webmap/viewer.html?webmap=" + SearchDict[itemSelection.value] + '>Open Item in Viewer</a>'
        elif gis.content.get(SearchDict[itemSelection.value]).type == "Story Map":
            url_text.value = '<a target="_blank" rel="noopener noreferrer" href=' + "https://storymaps.arcgis.com/stories/" + SearchDict[itemSelection.value] + '>Open Item in Viewer</a>'
        elif gis.content.get(SearchDict[itemSelection.value]).type == "Web Mapping Application":
            url_text.value = '<a target="_blank" rel="noopener noreferrer" href=' + "https://bcgov03.maps.arcgis.com/apps/webappviewer/index.html?id=" + SearchDict[itemSelection.value] + '>Open Item in Viewer</a>'
        elif gis.content.get(SearchDict[itemSelection.value]).type == "Dashboard":
            url_text.value = '<a target="_blank" rel="noopener noreferrer" href=' + "https://bcgov03.maps.arcgis.com/apps/dashboards/" + SearchDict[itemSelection.value] + '>Open Item in Viewer</a>' 

        if gis.content.get(SearchDict[itemSelection.value]).get_thumbnail():
            thumb_image.visibility = 'visible'
            thumb_image.value = gis.content.get(SearchDict[itemSelection.value]).get_thumbnail()
        else:
            thumb_image.visibility = 'hidden'
    else:
        bottom_box.children = ()
        title_text.value = "Title: "
        type_text.value = "Type: "
        owner_text.value = "Owner: "
        created_text.value = "Created: "
        modified_text.value = "Modified: "
        itemurl_text.value = None
        url_text.value = None
        thumb_image.value = None
        thumb_image.visibility = 'hidden'

### 4. Build Dictionaries for Old/Deprecated URLs and Old/Deprecated ItemIDs Lookup

In [4]:
# Read csv file from GITHub using pandas
url = 'https://raw.githubusercontent.com/MichaelDykesBC/AGO/master/lookup_maxl_to_minimap.csv'
df = pd.read_csv(url,index_col=0,encoding='cp1252')

# Empty Dictionaries to hold data
WHSE_Dict = {}
OldUrl_Dict = {}

# Iterate through CSV lookup table
for index, row in df.iterrows():
    if isinstance(row[4], str):
        OldServiceURL = os.path.join(*os.path.normpath(row[4]).split(os.path.sep)[5:])
        AGOTitle = row[6]
        if isinstance(row[8], str):
            AGOServiceURL = row[8]
            AGOItemID = row[1]
            WHSE_Name = row[7]
        elif isinstance(row[11], str):
            AGOServiceURL = row[11]
            AGOItemID = row[9]
            WHSE_Name = row[7]
        else:
            AGOServiceURL = None 
            AGOItemID = None
            WHSE_Name = row[7]
        
        WHSE_Dict[WHSE_Name] = [AGOTitle,AGOServiceURL,AGOItemID]
        OldUrl_Dict[OldServiceURL] = [AGOTitle,AGOServiceURL,AGOItemID]

### 5. Build Application/GUI

In [5]:
# Search AGO Groups
itemtypes_list = ["Web Map","Web Mapping Application","StoryMap","Dashboard"]

groups_list = []
groups = gis.groups.search(query = '*')
for group in groups:
    num_items = 0
    for content in group.content():
        if content.type in itemtypes_list:
            num_items += 1
    
    if num_items > 0:
        groups_list.append([group.title,group.id])

# Create dictionary for selection window group title + group id to access groups based on selection
GroupSearchDict = {}
for row in groups_list:
    GroupSearchDict[row[0]] = row[1]                           
        
# Search through AGO for content owned by whomever is running this Notebook (max 5000 items) and put them in a list object
item_list = []

for item in gis.content.search(query="* AND \  owner:" + gis.users.me.username, max_items=AGO_Max_Items):
    if item.type in itemtypes_list:
        item_list.append([item.title,item.id])

# Sort list object of AGO items alphabetically by title
sorteditem_list = sorted(item_list,key = lambda x:x[0])

# Create dictionary for selection window title + itemID to access items based on selection
SearchDict = {}
for row in sorteditem_list:
    SearchDisplay = row[0] + " | " + gis.content.get(row[1]).type
    SearchDict[SearchDisplay] = row[1]

# GUI building below
groupSelection = widgets.Dropdown(options=GroupSearchDict.keys(),value=None,description='Group:')
groupSelection.layout.width = "500px"
groupSelection.observe(group_content_selection_change,names='value') 

ownSelection = widgets.Button(description='All My Content')
ownSelection.on_click(own_content_selection)

itemSelection = widgets.Select(options=sorted(SearchDict.keys()),disabled=False)
itemSelection.layout.height = "350px"
itemSelection.layout.width = "700px" 
thumb_image = widgets.Image(value=gis.content.get(sorteditem_list[0][1]).get_thumbnail(),format="png",width=300,height=400)
title_text = widgets.Label(value="Title: " + sorteditem_list[0][0])
type_text = widgets.Label(value="Type: " + gis.content.get(sorteditem_list[0][1]).type) 
owner_text = widgets.Label(value="Owner: " + gis.content.get(sorteditem_list[0][1]).owner) 
created_text = widgets.Label(value="Created: " + str(time.strftime('%Y-%m-%d %H:%M', time.localtime(gis.content.get(sorteditem_list[0][1]).created/1000))))
modified_text = widgets.Label(value="Modified: " + str(time.strftime('%Y-%m-%d %H:%M', time.localtime(gis.content.get(sorteditem_list[0][1]).modified/1000))))
itemurl_text = widgets.HTML(value='<a target="_blank" rel="noopener noreferrer" href=' + gis.url + "/home/item.html?id=" + SearchDict[itemSelection.value] + '>Go to Item Page</a>')

if gis.content.get(sorteditem_list[0][1]).type == "Web Map":
    url_text = widgets.HTML(value='<a target="_blank" rel="noopener noreferrer" href=' + "https://bcgov03.maps.arcgis.com/home/webmap/viewer.html?webmap=" + SearchDict[itemSelection.value] + '>Open Item in Viewer</a>')
elif gis.content.get(sorteditem_list[0][1]).type == "Story Map":
    url_text = widgets.HTML(value='<a target="_blank" rel="noopener noreferrer" href=' + "https://storymaps.arcgis.com/stories/" + SearchDict[itemSelection.value] + '>Open Item in Viewer</a>')
elif gis.content.get(sorteditem_list[0][1]).type == "Web Mapping Application":
    url_text = widgets.HTML(value='<a target="_blank" rel="noopener noreferrer" href=' + "https://bcgov03.maps.arcgis.com/apps/webappviewer/index.html?id=" + SearchDict[itemSelection.value] + '>Open Item in Viewer</a>')
elif gis.content.get(sorteditem_list[0][1]).type == "Dashboard":
    url_text = widgets.HTML(value='<a target="_blank" rel="noopener noreferrer" href=' + "https://bcgov03.maps.arcgis.com/apps/dashboards/" + SearchDict[itemSelection.value] + '>Open Item in Viewer</a>')
      
itemSelection.observe(on_selection_change,names='value')  
Checkbutton = widgets.Button(description='Check Item')
Checkbutton.on_click(check_button_click)

Restorebutton = widgets.Button(description='Restore Item JSON')
Restorebutton.on_click(restore_button_click)

content_box = widgets.HBox([groupSelection,ownSelection])
info_box = widgets.VBox([title_text,type_text,owner_text,created_text,modified_text,itemurl_text,url_text])
top_box = widgets.HBox([thumb_image,info_box])
buttons_box = widgets.HBox([Checkbutton,Restorebutton])
bottom_box = widgets.VBox()
widgetlist = [content_box,itemSelection,top_box,buttons_box,bottom_box]
itemGUI = widgets.VBox(widgetlist)

## BCGW Service URL Updating Application

In [6]:
display(itemGUI)